In [19]:
import sys
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, PowerTransformer
from sklearn import set_config; set_config(display='diagram')

sys.path.append("/Users/antonis/code/Ant-mel/legendary_game_recs/")
from preprocessing.preprocess_1_cleaning import *
from preprocessing.preprocess_2_features import *
from preprocessing.pipeline_the_unification import *
from preprocessing.api_processing import *
from sklearn.compose import ColumnTransformer
from model.model_create import create_baseline_model, predict_baseline_model

from sklearn.neighbors import KNeighborsRegressor


In [37]:
ref_df = pd.read_json('../raw_data/reference_df_v1', orient='records', lines=True)
# x_train = ref_df = pd.read_json('../raw_data/xtrain_df_v1', orient='records', lines=True)

In [38]:
x_train.shape

(36850, 29)

In [39]:
ref_df

,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,genres,platforms,...,0,1,2,3,4,5,6,7,8,9
0,Darling Duality: Winter Wish,2024-12-31,7,0,2,2,['Melancholy Marionette'],2.2,[Visual Novel],"[Windows PC, Mac, Linux]",...,0,0,0,0,0,0,0,0,0,1
1,Anger Foot,2024-12-31,80,1,82,297,"['Devolver Digital', 'Robbie Fraser']",3.3,"[Brawler, Indie, Shooter]",[Windows PC],...,1,0,0,0,0,0,0,0,0,0
2,Life Is Feudal: MMO,2023-12-31,9,0,5,1,['Long Tale Games'],2.4,"[Adventure, RPG, Simulator]",[Windows PC],...,0,0,0,0,0,1,0,0,0,0
3,The Making of Karateka,2023-08-29,10,3,9,41,['Digital Eclipse'],4.5,"[Adventure, Fighting, Platform]","[Windows PC, PlayStation 4, Xbox One, PlayStat...",...,0,0,0,0,0,0,0,1,0,0
4,Enigma of Fear,2023-12-31,16,0,26,155,['Dumativa Game Studio'],2.1,"[Adventure, Indie, Puzzle]",[Windows PC],...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36845,Final Crisis: Terrestrial Defense Police,1991-12-01,4,0,1,4,['Techno Grard'],3.8,[Shooter],[PC-8801],...,0,0,1,0,0,0,0,0,0,0
36846,Level Up!,2009-11-25,6,0,0,1,[''],2.7,"[Adventure, Indie]",[Web browser],...,0,1,0,0,0,0,0,0,0,0
36847,Sea Battle,2014-02-07,3,0,0,0,['BYRIL'],2.2,[],"[Android, iOS]",...,0,0,0,0,0,0,0,0,0,1
36848,We Are Illuminati,2017-11-02,18,0,0,0,['Tapps Games'],2.0,"[Indie, Simulator, Strategy]","[Android, iOS]",...,1,0,0,0,0,0,0,0,0,0


In [40]:
api_cols_to_drop = ['game_engines', 'aggregated_rating', 'rating',
                    'aggregated_rating_count', 'age_ratings', 'multiplayer_modes']
api_1 = pd.read_json('../raw_data/igdb_api/api_json_0to30k')
api_2 = pd.read_json('../raw_data/igdb_api/api_json_30kto40k')
api_3 = pd.read_json('../raw_data/igdb_api/api_json_40kto50k')
api_4 = pd.read_json('../raw_data/igdb_api/api_json_50kto60k')
api_5 = pd.read_json('../raw_data/igdb_api/api_json_60kto90k')
api_6 = pd.read_json('../raw_data/igdb_api/api_json_90kto100k')
api_7 = pd.read_json('../raw_data/igdb_api/api_json_100kto110k')
api_8 = pd.read_json('../raw_data/igdb_api/api_json_110ktoEndk')
api_missed = pd.read_json('../raw_data/api_json_missed_retry_1')

api_data = pd.concat((api_1, api_2, api_3, 
                      api_4, api_5, api_6, 
                      api_7, api_8, api_missed), ignore_index=True)

api_data

,id,game_modes,name,themes,player_perspectives,game_engines,age_ratings,multiplayer_modes,aggregated_rating,aggregated_rating_count,rating
0,123757,"[{'id': 1, 'name': 'Single player'}]",PilotXross,"[{'id': 1, 'name': 'Action'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,106694,"[{'id': 1, 'name': 'Single player'}]",Plague Hunter,"[{'id': 1, 'name': 'Action'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,259874,NaN,The God Slayer,"[{'id': 1, 'name': 'Action'}, {'id': 38, 'name...","[{'id': 2, 'name': 'Third person'}]",NaN,NaN,NaN,NaN,NaN,NaN
3,211568,"[{'id': 1, 'name': 'Single player'}]",Rise of Rebellion,"[{'id': 1, 'name': 'Action'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,214912,"[{'id': 1, 'name': 'Single player'}]",Penny Blood,"[{'id': 17, 'name': 'Fantasy'}, {'id': 19, 'na...","[{'id': 2, 'name': 'Third person'}]",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
108399,40661,NaN,Polo,NaN,NaN,NaN,NaN,[18789],NaN,NaN,NaN
108400,245588,NaN,The Amazing Spider-Man Rescue,NaN,"[{'id': 4, 'name': 'Side view'}]",NaN,NaN,NaN,NaN,NaN,NaN
108401,2754,"[{'id': 1, 'name': 'Single player'}]",Breakout,"[{'id': 1, 'name': 'Action'}, {'id': 18, 'name...","[{'id': 4, 'name': 'Side view'}]",NaN,"[{'id': 707, 'rating': 8}]",NaN,NaN,NaN,79.008795
108402,18584,"[{'id': 1, 'name': 'Single player'}, {'id': 2,...",Stunt Cycle,NaN,"[{'id': 4, 'name': 'Side view'}]",NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
perspectives = api_data[['player_perspectives', 'id']]

In [42]:
merged =ref_df.merge(perspectives, left_on='game_id', right_on='id', how='left')

In [43]:
merged['player_perspectives'] = merged['player_perspectives'].apply(get_names_from_dict)

In [45]:
merged['player_perspectives'].fillna('', inplace=True)

In [46]:
enco, mlb = categorical_encoder(merged['player_perspectives'])

In [51]:
final = merged.merge(enco, left_index=True, right_index=True)

In [70]:
droped_a_few = drop_unnecesary_coulumns(final)

In [71]:
droped_a_few

,plays,playing,backlogs,wishlist,avg_review,total_reviews,total_lists,Adventure,Indie,RPG,...,9,player_perspectives,id,Auditory,Bird view / Isometric,First person,Side view,Text,Third person,Virtual Reality
0,7,0,2,2,2.2,1,4,0,0,0,...,1,[Text],178780.0,0,0,0,0,1,0,0
1,80,1,82,297,3.3,8,104,0,1,0,...,0,,NaN,0,0,0,0,0,0,0
2,9,0,5,1,2.4,0,3,1,0,1,...,0,"[First person, Third person]",24611.0,0,0,1,0,0,1,0
3,10,3,9,41,4.5,5,12,1,0,0,...,0,[Side view],256293.0,0,0,0,1,0,0,0
4,16,0,26,155,2.1,12,27,1,1,0,...,0,,NaN,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36845,4,0,1,4,3.8,0,2,0,0,0,...,0,,NaN,0,0,0,0,0,0,0
36846,6,0,0,1,2.7,1,2,1,1,0,...,0,,95373.0,0,0,0,0,0,0,0
36847,3,0,0,0,2.2,0,0,0,0,0,...,1,,134381.0,0,0,0,0,0,0,0
36848,18,0,0,0,2.0,2,7,0,1,0,...,0,,120333.0,0,0,0,0,0,0,0


In [81]:
X_train_prep = droped_a_few.drop(['id', 'player_perspectives'], axis=1)

In [82]:
X_train_prep

,plays,playing,backlogs,wishlist,avg_review,total_reviews,total_lists,Adventure,Indie,RPG,...,7,8,9,Auditory,Bird view / Isometric,First person,Side view,Text,Third person,Virtual Reality
0,7,0,2,2,2.2,1,4,0,0,0,...,0,0,1,0,0,0,0,1,0,0
1,80,1,82,297,3.3,8,104,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,9,0,5,1,2.4,0,3,1,0,1,...,0,0,0,0,0,1,0,0,1,0
3,10,3,9,41,4.5,5,12,1,0,0,...,1,0,0,0,0,0,1,0,0,0
4,16,0,26,155,2.1,12,27,1,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36845,4,0,1,4,3.8,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36846,6,0,0,1,2.7,1,2,1,1,0,...,0,0,0,0,0,0,0,0,0,0
36847,3,0,0,0,2.2,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
36848,18,0,0,0,2.0,2,7,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [83]:
num_transformer_yeo = PowerTransformer(method='yeo-johnson', standardize=False)

col_transformer_yeo = ColumnTransformer([('num_transformer', num_transformer_yeo,
                                   ['plays','playing','backlogs','wishlist','total_reviews','total_lists'])],
                                  remainder='passthrough')



In [84]:
transformed = pd.DataFrame(col_transformer_yeo.fit_transform(X_train_prep))

transformed

X_train = transformed.drop(6, axis=1)
y_train = transformed[6]

In [85]:
knn_model = KNeighborsRegressor(n_neighbors=11).fit(X_train, y_train)

In [88]:
final[(final['title'] == 'Call of Duty: Black Ops 4') 
               | (final['title'] == 'The Legend of Zelda: Breath of the Wild')
              | (final['title'] == 'Omori')
              | (final['title'] == 'FIFA 21')
              | (final['title'] == 'Tetris')
              | (final['title'] == 'Sid Meier\'s Civilization V')
              | (final['title'] == 'Red Dead Redemption')
              | (final['title'] == 'Grand Theft Auto V')
                 | (final['title'] == 'Pikmin 4')
              | (final['title'] == 'The Sims 4')
              | (final['title'] == 'Animal Crossing: New Horizons')]

,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,genres,platforms,...,9,player_perspectives,id,Auditory,Bird view / Isometric,First person,Side view,Text,Third person,Virtual Reality
33,Red Dead Redemption,2023-08-17,21,5,9,30,"['Double Eleven', 'Rockstar Games']",3.8,"[Adventure, RPG, Shooter]","[PlayStation 4, Nintendo Switch]",...,0,[Third person],260737.0,0,0,0,0,0,1,0
76,Pikmin 4,2023-07-21,1700,586,1200,2200,['Nintendo'],4.4,"[Adventure, Real Time Strategy, Strategy]",[Nintendo Switch],...,0,"[Third person, Bird view / Isometric]",59843.0,0,1,0,0,0,1,0
1752,Grand Theft Auto V,2022-06-15,45,5,7,2,[''],4.2,[],"[PlayStation 4, Xbox One, PlayStation 5, Xbox ...",...,0,,239064.0,0,0,0,0,0,0,0
4533,Omori,2020-12-25,11000,1300,6400,5000,"['OMOCAT', 'PLAYISM']",4.1,"[Adventure, Indie, RPG, Turn Based Strategy]","[Windows PC, Mac, PlayStation 4, Xbox One, Nin...",...,0,,NaN,0,0,0,0,0,0,0
5037,FIFA 21,2020-10-08,1500,39,56,19,"['Electronic Arts', 'EA Vancouver']",2.5,"[Simulator, Sport]","[Windows PC, PlayStation 4, Xbox One, PlayStat...",...,0,"[Third person, Bird view / Isometric]",134101.0,0,1,0,0,0,1,0
6113,Animal Crossing: New Horizons,2020-03-19,26000,2100,1800,1500,"['Nintendo EPD', 'Nintendo']",3.6,[Simulator],[Nintendo Switch],...,0,[Bird view / Isometric],109462.0,0,1,0,0,0,0,0
8746,Call of Duty: Black Ops 4,2018-10-12,4900,21,404,141,"['Treyarch', 'Activision']",2.3,[Shooter],"[Windows PC, PlayStation 4, Xbox One]",...,0,,NaN,0,0,0,0,0,0,0
11707,The Legend of Zelda: Breath of the Wild,2017-03-03,42000,3500,6600,3400,"['Nintendo EPD Production Group No. 3', 'Ninte...",4.4,"[Adventure, Puzzle, RPG]","[Wii U, Nintendo Switch]",...,0,[Third person],7346.0,0,0,0,0,0,1,0
16013,The Sims 4,2014-09-02,12000,640,1000,136,"['Electronic Arts', 'The Sims Studio']",3.1,"[RPG, Simulator]","[Windows PC, Mac, PlayStation 4, Xbox One, Pla...",...,0,"[First person, Third person, Bird view / Isome...",3212.0,0,1,1,0,0,1,0
17304,Grand Theft Auto V,2013-09-17,42000,1200,4600,895,"['Rockstar North', 'Rockstar Games']",3.8,"[Adventure, Racing, Shooter]","[Windows PC, Xbox 360, PlayStation 4, PlayStat...",...,0,"[First person, Third person]",1020.0,0,0,1,0,0,1,0


In [97]:
predict_baseline_model(8746, knn_model, final, X_train)

,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,genres,platforms,...,9,player_perspectives,id,Auditory,Bird view / Isometric,First person,Side view,Text,Third person,Virtual Reality
8746,Call of Duty: Black Ops 4,2018-10-12,4900,21,404,141,"['Treyarch', 'Activision']",2.3,[Shooter],"[Windows PC, PlayStation 4, Xbox One]",...,0,,NaN,0,0,0,0,0,0,0
11172,Arms,2017-06-16,3600,21,217,218,"['Nintendo of America', 'Nintendo EPD']",2.9,[Fighting],[Nintendo Switch],...,0,,NaN,0,0,0,0,0,0,0
4755,Call of Duty: Black Ops Cold War,2020-11-13,4400,144,556,283,"['Activision', 'Treyarch']",3.1,[Shooter],"[Windows PC, PlayStation 4, Xbox One, PlayStat...",...,0,[First person],137001.0,0,0,1,0,0,0,0
10451,Call of Duty: WWII,2017-11-02,6800,44,808,240,"['Sledgehammer Games', 'Activision']",2.8,[Shooter],"[Windows PC, PlayStation 4, Xbox One]",...,0,[First person],28204.0,0,0,1,0,0,0,0
5900,Predator: Hunting Grounds,2020-04-24,292,6,157,30,['Illfonic'],2.3,[Shooter],"[Windows PC, PlayStation 4]",...,0,,118272.0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15101,Rodea the Sky Soldier,2015-04-02,72,4,142,83,"['Kadokawa Games', 'Prope']",3.3,[Platform],[Wii],...,0,,138792.0,0,0,0,0,0,0,0
3810,PowerWash Simulator,2021-05-19,3200,370,499,479,"['Square Enix', 'FuturLab']",3.7,"[Indie, Simulator]","[Windows PC, PlayStation 4, Xbox One, PlayStat...",...,0,[First person],138590.0,0,0,1,0,0,0,0
20960,Alpha Protocol,2010-05-28,1200,12,635,241,"['Obsidian Entertainment', 'Sega']",3.3,"[RPG, Shooter]","[Windows PC, Xbox 360, Linux, PlayStation 3]",...,0,[Third person],2539.0,0,0,0,0,0,1,0
26019,Star Wars: Republic Commando,2005-02-17,2800,61,961,277,"['LucasArts', 'Activision']",3.6,"[Adventure, Shooter, Tactical]","[Windows PC, Xbox, PlayStation 4, Nintendo Swi...",...,0,[First person],148.0,0,0,1,0,0,0,0


In [98]:
from joblib import dump

final.to_json('../raw_data/reference_df_v3', orient='records', lines=True)
X_train.to_json('../raw_data/xtrain_df_v3', orient='records', lines=True)
dump(knn_model, '../raw_data/model_v3.joblib')

['../raw_data/model_v3.joblib']

In [15]:
def remove_link_trail(string):
    update = string.replace('https://www.backloggd.com', '')
    return update

In [33]:
api_cols_to_drop = ['game_engines', 'aggregated_rating', 'rating',
                    'aggregated_rating_count', 'age_ratings', 'multiplayer_modes']

extra_cols_to_drop = ['link', 'game_id', 'id', 'game_modes', 
                      'name', 'themes', 'player_perspectives']

In [34]:
def get_category_descriptions(json_dict):
    category_list = []
    
    if type(json_dict) == float:
        category_list = None
    else:
        for i in range(len(json_dict)):
            try:
                descriptions = json_dict[i]['content_descriptions']
                for i in range(len(descriptions)):
                    rating_descript = descriptions[i]['category']
                    category_list.append(rating_descript)
                    
            except:
                pass
    
    return category_list

def get_names_from_dict(dict_list):
    list_of_items = []
    try:
        for i in range(len(dict_list)):
            item = dict_list[i]['name']
            list_of_items.append(item)
    except:
        list_of_items = None
    
    return list_of_items

def prepare_json_df(original_json_df):
    
    original_json_df['themes'] = original_json_df['themes'].apply(get_names_from_dict)
    original_json_df['game_modes'] = original_json_df['game_modes'].apply(get_names_from_dict)
    original_json_df['player_perspectives'] = original_json_df['player_perspectives'].apply(get_names_from_dict)
    original_json_df['game_engines'] = original_json_df['game_engines'].apply(get_names_from_dict)
    original_json_df['age_ratings'] = original_json_df['age_ratings'].apply(get_category_descriptions)
    
    return original_json_df

In [35]:
def encode_api_data(raw_data, link_data, api_data):
    reference = pipeline_genre_ohe_only(raw_data)
    reference['url'] = reference['url'].apply(remove_link_trail)
    
    data_with_id = reference.merge(link_data, left_on='url', right_on='link')
    all_data = data_with_id.merge(api_data, left_on='game_id', right_on='id')
    
    prepped_data = prepare_json_df(all_data)    
    nec_data = prepped_data.drop(api_cols_to_drop, axis=1)
    filled_data = nec_data.fillna('')
    
    ohe_modes, mlb = categorical_encoder(filled_data['game_modes'])
    ohe_theme= keeping_ohe_columns_and_dropping_the_originals(filled_data[['themes']], 16)
    ohe_perspect, mlb = categorical_encoder(filled_data['player_perspectives'])
    
    ohe_colums = pd.concat((ohe_modes, ohe_theme, ohe_perspect), axis=1)
    
    everything = pd.concat((filled_data, ohe_colums), axis=1)
    
    return everything.drop(extra_cols_to_drop, axis=1)
    

In [36]:
prepped_api_data = prepare_json_df(api_data)

In [37]:
new_csv = pd.read_csv('../raw_data/reference_csv_v1')

In [39]:
prepped_api_data.merge(new_csv, left_on='id', right_on='game_id', how='right')['id'].isna().sum()

3167

In [9]:
num_transformer_yeo = PowerTransformer(method='yeo-johnson', standardize=False)

col_transformer = ColumnTransformer([('num_transformer', num_transformer_yeo,
                                   ['plays','playing','backlogs','wishlist','total_reviews','total_lists'])],
                                  remainder='passthrough')

def make_training_data_box_yeo(reference_data):
    dropped = drop_unnecesary_coulumns(reference_data)

    transformed = pd.DataFrame(col_transformer.fit_transform(dropped))

    X_train = transformed.drop(6, axis=1)
    y_train = transformed[6]

    return X_train, y_train

# Baseline model

In [8]:
game_data = pd.read_csv('../raw_data/all_game_data_v1_corrected2')
links_id = pd.read_csv('../raw_data/all_links_with_game_id_v1')

In [9]:
baseline_model, baseline_reference, X_train_baseline = create_baseline_model(game_data)

/Users/antonis/code/Ant-mel/legendary_game_recs/preprocessing/preprocess_1_cleaning.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column]=pd.to_datetime(df[column])
/Users/antonis/code/Ant-mel/legendary_game_recs/preprocessing/preprocess_2_features.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dupe['gen'] = None
/Users/antonis/code/Ant-mel/legendary_game_recs/preprocessing/preprocess_1_cleaning.py:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

In [16]:
predict_baseline_model(12935, baseline_model, baseline_reference, X_train_baseline)

,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,platforms,description,...,Simulator,Puzzle,Shooter,Strategy,Platform,Arcade,Sport,gen_1,gen_2,gen_3
12935,Pokémon Go,2016-07-06,17000,746,267,24,"[The Pokémon Company, Niantic Labs]",2.7,"[Android, iOS]",Travel between the real world and the virtual ...,...,1,0,0,0,0,0,0,0,0,1
16006,The Sims 4,2014-09-02,12000,640,1000,136,"[Electronic Arts, The Sims Studio]",3.1,"[Windows PC, Mac, PlayStation 4, Xbox One, Pla...",Unleash your imagination and create a unique w...,...,1,0,0,0,0,0,0,0,0,1
16664,Granblue Fantasy,2014-03-10,734,137,64,46,"[Cygames, Mobage]",3.2,"[Android, iOS]","The Japanese game got a patch on April 11, 201...",...,1,0,0,0,0,0,0,0,0,1
9421,State of Decay 2,2018-05-22,889,31,173,91,"[Undead Labs, Microsoft Studios]",3.1,"[Windows PC, Xbox One]",The next installment in the critically acclaim...,...,1,0,0,0,0,0,0,0,0,1
16706,Story of Seasons,2014-02-27,319,16,142,77,"[Marvelous AQL, XSEED Games]",3.5,[Nintendo 3DS],Story of Seasons is the latest Harvest Moon ti...,...,1,0,0,0,0,0,0,0,0,1
12989,Story of Seasons: Trio of Towns,2016-06-23,261,20,139,58,"[Marvelous, XSEED Games]",3.8,[Nintendo 3DS],The series is celebrating its 20th anniversary...,...,1,0,0,0,0,0,0,0,0,1
8595,Epic Seven,2018-11-08,223,35,14,4,"[Super Creative, Smilegate Megaport]",2.9,"[Android, Mac, iOS]","With Epic Seven's outstanding illustrations, i...",...,1,0,0,0,0,0,0,0,0,1
3449,Tears of Themis,2021-07-29,150,66,24,20,"[HoYoverse, Cognosphere]",3.6,"[Android, iOS]","As a new attorney at law, how will you solve a...",...,1,0,0,0,0,0,0,0,0,1
9580,The Idolmaster: Shiny Colors,2018-04-24,138,25,11,18,[Bandai Namco Entertainment],3.8,"[Android, iOS, Web browser]","The Idolmaster: Shiny Colors or ""Shinymas"", is...",...,1,0,0,0,0,0,0,0,0,1
14142,RPG Maker MV,2015-10-23,158,14,35,13,"[Degica Games, Kadokawa Games]",3.7,"[Windows PC, Mac, Linux, PlayStation 4, Xbox O...",POWERFUL enough for a developer SIMPLE enough ...,...,1,0,0,0,0,0,0,0,0,1


In [6]:
zelda = X_train_baseline[11702:11703]
wipeout = X_train_baseline[31357:31358]
omori = X_train_baseline[32334:32335]

In [7]:
ind_list = list(baseline_model.kneighbors(zelda,n_neighbors=11)[1][0])
baseline_reference.iloc[ind_list]

,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,platforms,description,...,Simulator,Puzzle,Shooter,Strategy,Platform,Arcade,Sport,gen_1,gen_2,gen_3
11702,The Legend of Zelda: Breath of the Wild,2017-03-03,42000,3500,6600,3400,"[Nintendo EPD Production Group No. 3, Nintendo]",4.4,"[Wii U, Nintendo Switch]",The Legend of Zelda: Breath of the Wild is the...,...,0,1,0,0,0,0,0,0,0,1
2293,Elden Ring,2022-02-25,27000,4700,7300,6400,"[FromSoftware, Bandai Namco Entertainment]",4.5,"[Windows PC, PlayStation 4, Xbox One, PlayStat...","Elden Ring is a fantasy, action and open world...",...,0,0,0,0,0,0,0,0,0,1
15141,Bloodborne,2015-03-24,23000,1400,7900,4600,"[FromSoftware, Sony Computer Entertainment]",4.5,[PlayStation 4],An action RPG in which the player embodies a H...,...,0,0,0,0,0,0,0,0,0,1
15353,Life is Strange,2015-01-29,20000,285,2900,1100,"[Feral Interactive, ""DONT NOD]",3.5,"[Windows PC, Android, Mac, Xbox 360, Linux, Pl...",Life is Strange is a five part episodic game t...,...,0,1,0,0,0,0,0,0,0,1
13475,Dark Souls III,2016-03-24,26000,1200,6200,3100,"[Bandai Namco Entertainment, FromSoftware]",4.3,"[Windows PC, PlayStation 4, Xbox One]",Dark Souls continues to push the boundaries wi...,...,0,0,0,0,0,0,0,0,0,1
14883,The Witcher 3: Wild Hunt,2015-05-19,23000,1600,9000,2100,"[CD Projekt RED, Spike ChunSoft]",4.3,"[Windows PC, PlayStation 4, Xbox One, PlayStat...","RPG and sequel to The Witcher 2 (2011), The Wi...",...,0,0,0,0,0,0,0,0,0,1
274,The Legend of Zelda: Tears of the Kingdom,2023-05-12,11000,6200,4700,6200,"[Nintendo, Nintendo EPD Production Group No. 3]",4.5,[Nintendo Switch],The Legend of Zelda: Tears of the Kingdom is t...,...,0,0,0,0,0,0,0,0,0,1
14354,Undertale,2015-09-15,37000,886,7100,2500,"[tobyfox, 8-4]",4.3,"[Windows PC, Mac, Linux, PlayStation 4, Xbox O...","A small child falls into the Underground, wher...",...,0,0,0,0,0,0,0,0,0,1
2430,Pokémon Legends: Arceus,2022-01-28,13000,1800,3700,2600,"[Nintendo, Game Freak]",3.7,[Nintendo Switch],The Pokémon Legends: Arceus game honors past P...,...,0,0,0,0,0,0,0,0,0,1
12568,Persona 5,2016-09-15,20000,747,3500,2000,"[Deep Silver, P Studio]",4.1,"[PlayStation 4, PlayStation 3]","Persona 5, a turn-based JRPG with visual novel...",...,0,0,0,0,0,0,0,0,0,1


In [11]:
data_processed_func = encode_api_data(game_data, links_id, api_data)

/Users/antonis/code/Ant-mel/legendary_game_recs/preprocessing/preprocess_1_cleaning.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column]=pd.to_datetime(df[column])
/Users/antonis/code/Ant-mel/legendary_game_recs/preprocessing/preprocess_2_features.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dupe['gen'] = None
/Users/antonis/code/Ant-mel/legendary_game_recs/preprocessing/preprocess_1_cleaning.py:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

### MinMax Scaled Data

In [13]:
X_train, y_train = make_training_data(data_processed_func)

In [14]:
X_train

,0,1,2,3,4,5,7,8,9,10,...,39,40,41,42,43,44,45,46,47,48
0,0.000130,0.000161,0.000167,0.000313,0.000263,0.000690,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.000174,0.000161,0.000417,0.000156,0.000000,0.000517,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.000174,0.000161,0.000417,0.000156,0.000000,0.000517,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,0.000196,0.000161,0.004000,0.013281,0.000526,0.005517,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.000043,0.000161,0.000000,0.001406,0.000000,0.000172,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35620,0.000065,0.000161,0.000167,0.001250,0.000000,0.000690,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
35621,0.000109,0.000161,0.000000,0.000156,0.000263,0.000345,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35622,0.000043,0.000161,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35623,0.000370,0.000161,0.000000,0.000000,0.000526,0.001207,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Yeo Johnson scaled data

In [15]:
X_train_yeo, y_train_yeo = make_training_data_box_yeo(data_processed_func)

## Predicting on MinMax

In [17]:
knn_model_minmax = KNeighborsRegressor(n_neighbors=11).fit(X_train, y_train)

In [31]:
zelda_mask = data_processed_func['title'] == "The Legend of Zelda: Breath of the Wild"
wipout_mask = data_processed_func['title'] == "Wipeout"
stardew_mask = data_processed_func['title'] == "Stardew Valley"

In [45]:
zelda = X_train[11399:11400]
wipeout = X_train[31357:31358]
stardew = X_train[13221:13222]


In [49]:
ind_list = list(knn_model_minmax.kneighbors(wipeout,n_neighbors=11)[1][0])

In [50]:
ind_list

[31357, 25559, 27658, 26112, 31001, 29597, 25177, 22788, 18555, 23001, 29745]

In [52]:
data_processed_func.columns

Index(['title', 'release_date', 'plays', 'playing', 'backlogs', 'wishlist',
       'developers', 'avg_review', 'platforms', 'description', 'total_reviews',
       'total_lists', 'main', 'ratings_zero_five', 'ratings_one_zero',
       'ratings_one_five', 'ratings_two_zero', 'ratings_two_five',
       'ratings_three_zero', 'ratings_three_five', 'ratings_four_zero',
       'ratings_four_five', 'ratings_five_zero', 'image', 'url', 'Adventure',
       'Indie', 'RPG', 'Simulator', 'Puzzle', 'Shooter', 'Strategy',
       'Platform', 'Arcade', 'Sport', 'gen_1', 'gen_2', 'gen_3',
       'Battle Royale', 'Co-operative', 'Massively Multiplayer Online (MMO)',
       'Multiplayer', 'Single player', 'Split screen', 'Action', 'Fantasy',
       'Science fiction', 'Comedy', 'Horror', 'Historical', 'Mystery',
       'Open world', 'Kids', 'Survival', 'Sandbox', 'Warfare', 'Party',
       'Drama', 'Educational', 'Erotic', 'Auditory', 'Bird view / Isometric',
       'First person', 'Side view', 'Text', 'Th

# Everything below was working on the above

In [7]:
reference['url'] = reference['url'].apply(remove_link_trail)

0        /games/darling-duality-winter-wish/
1                         /games/anger-foot/
2                 /games/life-is-feudal-mmo/
3             /games/the-making-of-karateka/
4                     /games/enigma-of-fear/
                        ...                 
36815                   /games/final-crisis/
36816                    /games/level-up--1/
36817                  /games/sea-battle--2/
36818              /games/we-are-illuminati/
36819                     /games/karate-cat/
Name: url, Length: 36820, dtype: object

In [8]:
data_with_id = reference.merge(links_id, left_on='url', right_on='link')

## Merging data_with_id to api data

In [10]:
all_data = data_with_id.merge(api_data, left_on='game_id', right_on='id')

In [ ]:
relevant_columns = all_data.drop(api_cols_to_drop, axis=1)

## Cetegorically encoding the api columns

In [14]:
prepped_data = prepare_json_df(all_data)

In [40]:
nec_data = prepped_data.drop(api_cols_to_drop, axis=1)
filled_data = nec_data.fillna('')

In [60]:
ohe_modes, mlb = categorical_encoder(filled_data['game_modes'])
ohe_theme= keeping_ohe_columns_and_dropping_the_originals(filled_data[['themes']], 16)
ohe_perspect, mlb = categorical_encoder(filled_data['player_perspectives'])

In [61]:
ohe_colums = pd.concat((ohe_modes, ohe_theme, ohe_perspect), axis=1)

In [67]:
pd.concat((filled_data, ohe_colums), axis=1)

,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,platforms,description,...,Drama,Educational,Erotic,Auditory,Bird view / Isometric,First person,Side view,Text,Third person,Virtual Reality
0,Darling Duality: Winter Wish,2024-12-31,7,0,2,2,[Melancholy Marionette],2.2,"[Windows PC, Mac, Linux]",We all have multiple sides. Pick your poison! ...,...,0,0,0,0,0,0,0,1,0,0
1,Life Is Feudal: MMO,2023-12-31,9,0,5,1,[Long Tale Games],2.4,[Windows PC],Life is Feudal: MMO is an open-world survival ...,...,0,0,0,0,0,1,0,0,1,0
2,Life Is Feudal: MMO,2023-12-31,9,0,5,1,[Long Tale Games],2.4,[Windows PC],Life is Feudal: MMO is an open-world survival ...,...,0,0,0,0,0,1,0,0,1,0
3,They Speak From the Abyss,2023-12-31,10,0,48,85,"[2 Left Thumbs, Nikki Kalpa]",3.2,[Windows PC],Venture into the Abyss and face strange and ho...,...,0,0,0,0,0,1,0,0,0,0
4,Shred and Tear: Explosive Kajun,2023-12-31,3,0,0,9,"[Shady Corner Games, Lucid Realm Games]",1.5,[Windows PC],"An ultraviolent, fast-paced, sexy and gory Hac...",...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35620,Davigo,2023-12-31,4,0,2,8,[Davigo Studio],3.5,"[SteamVR, Oculus Quest 2]","DAVIGO is a VR vs. PC ""cross-reality"" battle g...",...,0,0,0,0,0,0,0,0,1,1
35621,Level Up!,2009-11-25,6,0,0,1,[],2.7,[Web browser],"Level Up! is a Game created by Titch007uk, and...",...,0,0,0,0,0,0,0,0,0,0
35622,Sea Battle,2014-02-07,3,0,0,0,[BYRIL],2.2,"[Android, iOS]",Sea Battle is a board game everybody loves sin...,...,0,0,0,0,0,0,0,0,0,0
35623,We Are Illuminati,2017-11-02,18,0,0,0,[Tapps Games],2.0,"[Android, iOS]","Control minds, insert subliminal messages into...",...,0,0,0,0,0,0,0,0,0,0
